In [4]:
from scipy.stats import ttest_ind
import scipy.stats
import numpy as np
import pandas as pd
import math
file = '../../intermediate-results-downsampling/nuc_mutations_by_gene.csv'
df = pd.read_csv(file)

In [5]:
stats_df = df.groupby(['Mutation', 'Host'])['Count'].agg(['mean', 'std']).reset_index()
n = 26 # number of genes
confidence = 0.95 # confidence interval 
stats_df = stats_df.assign(se=stats_df['std'] / math.sqrt(n))
h = stats_df['se'] * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
ci = h * 2
stats_df = stats_df.assign(confidence_interval=ci)                         
print(stats_df)

   Mutation   Host       mean        std        se  confidence_interval
0    A_to_C  Camel   1.692308   4.047411  0.793763             3.269570
1    A_to_C  Human   0.461538   0.760567  0.149159             0.614399
2    A_to_G  Camel   6.230769   8.538420  1.674522             6.897485
3    A_to_G  Human   2.346154   3.898126  0.764485             3.148974
4    A_to_T  Camel   1.923077   2.827339  0.554487             2.283974
5    A_to_T  Human   0.653846   1.164210  0.228320             0.940469
6    C_to_A  Camel   2.769231   3.178776  0.623409             2.567871
7    C_to_A  Human   0.884615   1.395046  0.273591             1.126943
8    C_to_G  Camel   1.038462   1.821664  0.357258             1.471572
9    C_to_G  Human   0.384615   0.752432  0.147564             0.607828
10   C_to_T  Camel  32.038462  33.391593  6.548630            26.974312
11   C_to_T  Human   9.500000   9.563472  1.875551             7.725540
12   G_to_A  Camel   7.269231   8.263451  1.620596             6

In [6]:
ttest_results = []

mutations = df['Mutation'].unique()
for mutation in mutations:
    camel_counts = df[(df['Mutation'] == mutation) & (df['Host'] == 'Camel')]['Count']
    human_counts = df[(df['Mutation'] == mutation) & (df['Host'] == 'Human')]['Count']
    
    t_stat, p_val = ttest_ind(camel_counts, human_counts, equal_var=False)  # Assuming unequal variances
    ttest_results.append({
        'Mutation': mutation,
        't_stat': t_stat,
        'p_val': p_val
    })

ttest_df = pd.DataFrame(ttest_results)
print(ttest_df)


   Mutation    t_stat     p_val
0    A_to_T  2.116603  0.041864
1    A_to_G  2.110312  0.042053
2    A_to_C  1.523879  0.139269
3    C_to_T  3.308680  0.002505
4    C_to_A  2.768229  0.009026
5    C_to_G  1.691563  0.100072
6    G_to_A  2.951628  0.005795
7    G_to_T  3.139197  0.003619
8    G_to_C  2.011368  0.051929
9    T_to_A  2.500782  0.018201
10   T_to_G  2.812631  0.008812
11   T_to_C  3.772550  0.000770
